In [1]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import gensim

import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import *
from utils.tm_mgmt import update_topic_scores
from utils.text import *
from django.db.models import Q, Count, Func, F, Sum, Value, Case, When, IntegerField

In [2]:
def clean_text(text):
    text = text.replace('\r', '\n')
    text = text.replace(u'\xa0', ' ')
    text = text.replace(u'\x96', '-')
    text = text.replace(u'\xad', '-')
    text = text.replace(u'\u2014', '–')
    # text = text.replace(u'\u2013', '–')
    text = text.replace('(', '')
    text = text.replace(')', '')
    return text

### Finding agenda items

Search id 89: Searches 84, 85, 88, 87 combined 
- 84: (01/01 - 13/85) updated - from https.*scans of pdfs with xml metadata
- 85: (13/86 - 18/210) GermaParlTEI
- 87: (19/01 - 19/108) updated - XML from www.bundestag.de/service/opendata
- 88: (18/211 - 18/245) from https.*scans of pdfs with xml metadata

Search id 96: Searches 90, 95, 92, 93 combined
- 90: (01/01 - 13/85) updated - from https.*scans of pdfs with xml metadata
- 92: (18/211 - 18/245) from https.*scans of pdfs with xml metadata
- 93: (19/01 - 19/108) updated - XML from www.bundestag.de/service/opendata
- 95: (13/86 - 18/210) updated - GermaParlTEI

In [8]:
# finding the agenda items of the utterances that appear in the search
utterances = pm.Utterance.objects.filter(search_matches__pk=89)
#agenda_items = pm.AgendaItem.objects.filter()

In [33]:
agenda_items = []
agenda_items_doc_date = []
for ut in utterances:
    try:
        agenda_items.append(ut.agenda_item.title)
        agenda_items_doc_date.append(ut.document.date)
    except AttributeError:
        count_missing += 1
        pass

print(len(agenda_items))
print(count_missing)

2018-03-23
2018-03-23
2018-03-23
2018-03-23
2018-03-23
2018-03-23
2018-03-23
2018-03-23
2018-04-25
2018-04-25
2018-04-25
2018-04-25
2018-04-25
2018-04-26
2018-04-26
2018-04-26
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-06-07
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-15
2018-06-15
2018-06-28
2018-06-28
2018-06-28
2018-06-28
2018-06-28
2018-06-28
2018-06-28
2018-06-28
2018-06-29
2018-06-29
2018-07-03
2018-07-03
2018-07-03
2018-07-03
2018-07-03
2018-07-03
2018-07-03
2018-07-03
2018-07-03
2018-07-03
2018-07-03
2018-07-03
2018-07-03
2018-07-03
2018-07-04
2018-07-04
2018-07-04
2018-07-04
2017-11-21
2017-11-21
2017-11-21
2017-11-21
2017-11-21
2017-11-21
2017-11-21
2017-11-21
2017-11-21
2017-11-21
2017-11-21
2017-11-21
2017-11-21
2017-12-12
2018-07-04
2018-07-04
2018-07-05
2017-12-12

In [18]:
df_agenda_items = pd.DataFrame(agenda_items)
df_agenda_items.columns = ['Agenda Item']

In [23]:
d = {'Agenda Item': agenda_items, 'Date': agenda_items_doc_date}
df_d = pd.DataFrame.from_dict(d)

In [25]:
df_d.sort_values(by=['Date'])

,Agenda Item,Date
51,Beratung des Antrags der Fraktion BÜNDNIS 90/D...,2017-11-21
31,Beratung des Antrags der Fraktion BÜNDNIS 90/D...,2017-11-21
32,Beratung des Antrags der Fraktion BÜNDNIS 90/D...,2017-11-21
33,Beratung des Antrags der Fraktion BÜNDNIS 90/D...,2017-11-21
34,Beratung des Antrags der Fraktion BÜNDNIS 90/D...,2017-11-21
35,Beratung des Antrags der Fraktion BÜNDNIS 90/D...,2017-11-21
36,Beratung des Antrags der Fraktion BÜNDNIS 90/D...,2017-11-21
37,Beratung des Antrags der Fraktion BÜNDNIS 90/D...,2017-11-21
38,Beratung des Antrags der Fraktion BÜNDNIS 90/D...,2017-11-21
39,Beratung des Antrags der Fraktion BÜNDNIS 90/D...,2017-11-21


In [19]:
# extract unique agenda items only 
agenda_items_unique = list(set(agenda_items))
df_agenda_items_unique = pd.DataFrame(agenda_items_unique)
df_agenda_items_unique.columns = ['Agenda Item']

In [29]:
pd.set_option('max_colwidth', 1000)
df_agenda_items_unique

,Agenda Item
0,"b) Beratung der Beschlussempfehlung und des Berichts des Ausschusses für Wirtschaft und Energie (9. Ausschuss) zu dem Antrag der Abgeordneten Lorenz Gösta Beutin, Ralph Lenkert, Hubertus Zdebel, weiterer Abgeordneter und der Fraktion DIE LINKE"
1,Erste Beratung des von dem Abgeordneten Roman Johannes Reusch und der Fraktion der AfD eingebrachten Entwurfs eines Gesetzes zur Änderung des Strafgesetzbuches und weiterer Gesetze ‒ Bekämpfung der Haushaltsuntreue und zur Sicherstellung der ordnungsgemäßen Verwendung öffentlicher Mittel
2,"hier: Einzelplan 16 Bundesministerium für Umwelt, Naturschutz und nukleare Sicherheit"
3,hier: Einzelplan 09 Bundesministerium für Wirtschaft und Energie
4,Beratung der Unterrichtung durch den Parlamentarischen Beirat für nachhaltige Entwicklung
5,"Beratung des Antrags der Abgeordneten Zaklin Nastic, Michel Brandt, Heike Hänsel, weiterer Abgeordneter und der Fraktion DIE LINKE"
6,CO 2 -Steuer und ihre Auswirkungen auf Energiepreise
7,24. Erste Beratung des von den Fraktionen der CDU/CSU und SPD eingebrachten Entwurfs eines Gesetzes zur Reform der Hebammenausbildung und zur Änderung des Fünften Buches Sozialgesetzbuch (Hebammenreformgesetz – HebRefG)
8,Abgabe einer Regierungserklärung durch die Bundeskanzlerin
9,– zu dem Antrag der Fraktion BÜNDNIS 90/DIE GRÜNEN


---

### Finding agenda items from TEI/pdf scraper

In [31]:
# filter utterances from specific document, then find agenda items (if any)
pdf_search = pm.Utterance.objects.filter(document__id=14274)
pdf_search_agenda = []
for ut in pdf_search:
    try:
        pdf_search_agenda.append(ut.agenda_item.title)
    except AttributeError:
        pass

In [32]:
pdf_search_agenda

[]

In [34]:
# filter for agenda items associated with a document
pdf_doc_tops = pm.AgendaItem.objects.filter(document__id=14272)
for tops in pdf_doc_tops:
    print(tops.title)

Ich rufe den Tagesordnungspunkt 2 auf:
Wahl des Präsidenten, verbunden mit Namensaufruf und Feststellung der Beschlußfähigkeit
Liebe Kolleginnen und Kollegen, ich rufe den Tagesordnungspunkt 4 auf:
Beschlußfassung über die
Ich rufe Tagesordnungspunkt 6 auf:
Wahl der Stellvertreter des Präsidenten


---

### Preparing corpus and dictionary

In [7]:
agenda_items_unique_lower = [word.lower() for word in agenda_items_unique]

In [8]:
agenda_items_unique_clean = [clean_text(text) for text in agenda_items_unique_lower]

In [9]:
from gensim import corpora
from nltk.corpus import stopwords
stop_words = set(stopwords.words('german'))

# Tokenize(split) the sentences into words
texts = [[text for text in doc.split()] for doc in agenda_items_unique_clean]

# Remove stopwords 
texts_no_sw = []
for text in texts:
    no_sw = [word for word in text if word not in stop_words]
    texts_no_sw.append(no_sw)
    

In [10]:
# Create dictionary
dictionary = corpora.Dictionary(texts_no_sw)

# Get information about the dictionary
print(dictionary)

Dictionary(400 unique tokens: ['9.', 'abgeordneten', 'abgeordneter', 'antrag', 'ausschuss']...)


In [11]:
agenda_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts_no_sw]

In [25]:
# show word counts

import urllib.request
from collections import Counter

import numpy as np 

from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

data = " ".join(agenda_items_unique_clean)

stopword_list = stopwords.words("german")

# Note that `ngram_range=(1, 1)` means we want to extract Unigrams, i.e. tokens.
ngram_vectorizer = CountVectorizer(analyzer='word', tokenizer=word_tokenize,
                                   ngram_range=(1, 1), min_df=1, stop_words=stopword_list)
# X matrix where the row represents sentences and column is our one-hot vector for each token in our vocabulary
X = ngram_vectorizer.fit_transform(data.split('\n'))

# Vocabulary
vocab = list(ngram_vectorizer.get_feature_names())

# Column-wise sum of the X matrix.
# It's some crazy numpy syntax that looks horribly unpythonic
# For details, see http://stackoverflow.com/questions/3337301/numpy-matrix-to-array
# and http://stackoverflow.com/questions/13567345/how-to-calculate-the-sum-of-all-columns-of-a-2d-numpy-array-efficiently
counts = X.sum(axis=0).A1

freq_distribution = Counter(dict(zip(vocab, counts)))
print (freq_distribution.most_common(100))

[(',', 169), ('beratung', 70), ('fraktion', 60), ('abgeordneten', 54), ('weiterer', 51), ('abgeordneter', 49), ('antrags', 46), ('dr.', 40), ('90/die', 22), ('bündnis', 22), ('grünen', 22), ('b', 16), ('fdp', 15), ('linke', 15), ('afd', 14), ('ausschuss', 14), ('beschlussempfehlung', 13), ('ausschusses', 12), ('antrag', 11), ('bundesregierung', 11), (':', 10), ('eingebrachten', 10), ('einzelplan', 10), ('entwurfs', 10), ('gesetzes', 10), ('berichts', 9), ('erste', 8), ('aggelidis', 7), ('grigorios', 7), ('–', 7), ('badum', 6), ('lisa', 6), ('naturschutz', 6), ('nukleare', 6), ('sicherheit', 6), ('umwelt', 6), ('bundesministerium', 5), ('c', 5), ('hoffmann', 5), ('16.', 4), ('alexander', 4), ('bericht', 4), ('bettina', 4), ('cdu/csu', 4), ('energie', 4), ('fraktionen', 4), ('spd', 4), ('unterrichtung', 4), ('verlangen', 4), ('wirtschaft', 4), ('2', 3), ('2019', 3), ('4.', 3), ('9.', 3), ('anja', 3), ('beutin', 3), ('bundeskanzlerin', 3), ('co', 3), ('ferschl', 3), ('frank', 3), ('franzi

### Running a simple topic model

In [12]:
# Step 0: Import packages and stopwords
from gensim.models import LdaModel, LdaMulticore
from gensim.utils import simple_preprocess, lemmatize
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)

In [13]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=agenda_corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

2019-08-11 16:45:14,305 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2019-08-11 16:45:14,308 : INFO : using symmetric eta at 0.1
2019-08-11 16:45:14,309 : INFO : using serial LDA version on this node
2019-08-11 16:45:14,318 : INFO : running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 99 documents, updating model once every 99 documents, evaluating perplexity every 99 documents, iterating 50x with a convergence threshold of 0.001000
2019-08-11 16:45:14,469 : INFO : -10.503 per-word bound, 1451.4 perplexity estimate based on a held-out corpus of 99 documents with 1214 words
2019-08-11 16:45:14,470 : INFO : PROGRESS: pass 0, at document #99/99
2019-08-11 16:45:14,591 : INFO : optimized alpha [0.09454434, 0.09445321, 0.09212606, 0.078220725, 0.080531895, 0.07877312, 0.07648808, 0.07261585, 0.08237724, 0.076027945]
2019-08-11 16:45:14,597 : INFO : topic #7 (0.073): 0.032*"beratung" + 0.017*"cdu/csu" +

2019-08-11 16:45:15,067 : INFO : topic #1 (0.082): 0.055*"beratung" + 0.054*"dr." + 0.054*"fraktion" + 0.051*"abgeordneter" + 0.051*"abgeordneten" + 0.051*"weiterer" + 0.041*"antrags" + 0.031*"afd" + 0.028*"b" + 0.019*"fdp"
2019-08-11 16:45:15,068 : INFO : topic #0 (0.085): 0.058*"fraktion" + 0.046*"grünen" + 0.046*"90/die" + 0.046*"bündnis" + 0.044*"dr." + 0.043*"beratung" + 0.040*"abgeordneter" + 0.040*"weiterer" + 0.040*"abgeordneten" + 0.030*"antrags"
2019-08-11 16:45:15,069 : INFO : topic diff=0.055881, rho=0.408248
2019-08-11 16:45:15,131 : INFO : -5.663 per-word bound, 50.7 perplexity estimate based on a held-out corpus of 99 documents with 1214 words
2019-08-11 16:45:15,132 : INFO : PROGRESS: pass 5, at document #99/99
2019-08-11 16:45:15,172 : INFO : optimized alpha [0.08382252, 0.08033282, 0.07262361, 0.05606041, 0.057044327, 0.058720324, 0.05190908, 0.044839364, 0.05767987, 0.049393546]
2019-08-11 16:45:15,174 : INFO : topic #7 (0.045): 0.019*"beratung" + 0.018*"cdu/csu" + 0

2019-08-11 16:45:15,688 : INFO : topic #1 (0.074): 0.056*"beratung" + 0.054*"dr." + 0.054*"fraktion" + 0.051*"abgeordneter" + 0.051*"abgeordneten" + 0.051*"weiterer" + 0.042*"antrags" + 0.032*"afd" + 0.028*"b" + 0.019*"fdp"
2019-08-11 16:45:15,690 : INFO : topic #0 (0.079): 0.059*"fraktion" + 0.048*"grünen" + 0.048*"90/die" + 0.048*"bündnis" + 0.043*"dr." + 0.043*"beratung" + 0.040*"abgeordneter" + 0.040*"weiterer" + 0.039*"abgeordneten" + 0.030*"antrags"
2019-08-11 16:45:15,691 : INFO : topic diff=0.016337, rho=0.301511


In [14]:
doc_lda = lda_model[agenda_corpus]

In [15]:
import pprint
pp = pprint.PrettyPrinter(indent=0)
pp.pprint(lda_model.print_topics())

2019-08-11 16:45:16,983 : INFO : topic #0 (0.079): 0.059*"fraktion" + 0.048*"grünen" + 0.048*"90/die" + 0.048*"bündnis" + 0.043*"dr." + 0.043*"beratung" + 0.040*"abgeordneter" + 0.040*"weiterer" + 0.039*"abgeordneten" + 0.030*"antrags"
2019-08-11 16:45:16,985 : INFO : topic #1 (0.074): 0.056*"beratung" + 0.054*"dr." + 0.054*"fraktion" + 0.051*"abgeordneter" + 0.051*"abgeordneten" + 0.051*"weiterer" + 0.042*"antrags" + 0.032*"afd" + 0.028*"b" + 0.019*"fdp"
2019-08-11 16:45:16,986 : INFO : topic #2 (0.065): 0.049*"abgeordneten" + 0.046*"beratung" + 0.046*"fraktion" + 0.041*"weiterer" + 0.039*"antrags" + 0.037*"abgeordneter" + 0.023*"dr." + 0.019*"linke" + 0.016*"grünen" + 0.016*"90/die"
2019-08-11 16:45:16,987 : INFO : topic #3 (0.049): 0.076*"bundesregierung" + 0.072*"beratung" + 0.047*"erste" + 0.047*"entwurfs" + 0.047*"eingebrachten" + 0.047*"gesetzes" + 0.020*"unterrichtung" + 0.011*"berichts" + 0.011*"beschlussempfehlung" + 0.010*"ausschusses"
2019-08-11 16:45:16,988 : INFO : topic 

[(0,
'0.059*"fraktion" + 0.048*"grünen" + 0.048*"90/die" + 0.048*"bündnis" + '
'0.043*"dr." + 0.043*"beratung" + 0.040*"abgeordneter" + 0.040*"weiterer" + '
'0.039*"abgeordneten" + 0.030*"antrags"'),
(1,
'0.056*"beratung" + 0.054*"dr." + 0.054*"fraktion" + 0.051*"abgeordneter" + '
'0.051*"abgeordneten" + 0.051*"weiterer" + 0.042*"antrags" + 0.032*"afd" + '
'0.028*"b" + 0.019*"fdp"'),
(2,
'0.049*"abgeordneten" + 0.046*"beratung" + 0.046*"fraktion" + 0.041*"weiterer" '
'+ 0.039*"antrags" + 0.037*"abgeordneter" + 0.023*"dr." + 0.019*"linke" + '
'0.016*"grünen" + 0.016*"90/die"'),
(3,
'0.076*"bundesregierung" + 0.072*"beratung" + 0.047*"erste" + 0.047*"entwurfs" '
'+ 0.047*"eingebrachten" + 0.047*"gesetzes" + 0.020*"unterrichtung" + '
'0.011*"berichts" + 0.011*"beschlussempfehlung" + 0.010*"ausschusses"'),
(4,
'0.050*"abgeordneten" + 0.043*"fraktion" + 0.040*"beratung" + '
'0.037*"abgeordneter" + 0.037*"weiterer" + 0.031*"antrags" + 0.022*"linke" + '
'0.020*"fdp" + 0.013*"beer," + 0.013*"n

### Visualising results of topic model

In [16]:
import pyLDAvis 
import pyLDAvis.gensim 
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, agenda_corpus, dictionary)
vis

ModuleNotFoundError: No module named 'pyLDAvis'